# Create DESeq2 master data sets

create a master count matrix and master colData data

ref: [sql-programming-guide.htm](https://spark.apache.org/docs/latest/sql-programming-guide.html)

In [1]:
SPARK_HOME="../../sparkBin/spark-3.1.2-bin-hadoop3.2"
import findspark
findspark.init( SPARK_HOME )

In [2]:
import pandas as pd
import pathlib as Path

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("DESeqMasterETL") \
    .getOrCreate()

# .config("spark.some.config.option", "some-value") \

In [4]:
from pyspark.sql import functions as sqlFunc

## load our mock terra data model

In [5]:
sampleFile = "testData/sample.tsv"
samplePDF = pd.read_csv(sampleFile, sep="\t")

In [6]:
fileLst = samplePDF.loc[:, 'quant.sf'].tolist()
fileLst

['ctrl.1.quant.sf',
 'ctrl.2.quant.sf',
 'ctrl.3.quant.sf',
 'kras.1.quant.sf',
 'kras.2.quant.sf',
 'kras.3.quant.sf']

In [7]:
# pre allocate slots to store data frames in
quantSDFs = [None] * len(fileLst)

In [8]:
quantSchema = "`Name` STRING, `Length` INT, `EffectiveLength` DOUBLE, `Tmp` DOUBLE, `NumReads` DOUBLE "

for i in range( len(quantSDFs) ):
    quantFile = "testData/{}".format( fileLst[i] )
    print(quantFile)
    df = spark.read.load( quantFile, format="csv", sep="\t", 
                             schema=quantSchema, header="true")
    quantSDFs[i] = df

testData/ctrl.1.quant.sf
testData/ctrl.2.quant.sf
testData/ctrl.3.quant.sf
testData/kras.1.quant.sf
testData/kras.2.quant.sf
testData/kras.3.quant.sf


In [9]:
quantSDFs[0].printSchema()

root
 |-- Name: string (nullable = true)
 |-- Length: integer (nullable = true)
 |-- EffectiveLength: double (nullable = true)
 |-- Tmp: double (nullable = true)
 |-- NumReads: double (nullable = true)



## explore quant.sf file format

In [10]:
quantSDFs[0].show()

+--------------------+------+---------------+---------+--------+
|                Name|Length|EffectiveLength|      Tmp|NumReads|
+--------------------+------+---------------+---------+--------+
|ENST00000456328.2...|  1657|         1530.0|      0.0|     0.0|
|ENST00000450305.2...|   632|          505.0|      0.0|     0.0|
|ENST00000488147.1...|  1351|         1224.0|      0.0|     0.0|
|ENST00000619216.1...|    68|           15.0|      0.0|     0.0|
|ENST00000473358.1...|   712|          585.0|      0.0|     0.0|
|ENST00000469289.1...|   535|          408.0|      0.0|     0.0|
|ENST00000607096.1...|   138|           55.0|      0.0|     0.0|
|ENST00000417324.1...|  1187|         1060.0|      0.0|     0.0|
|ENST00000461467.1...|   590|         885.84| 5.813169|    4.93|
|ENST00000606857.1...|   840|          713.0|      0.0|     0.0|
|ENST00000642116.1...|  1414|         1287.0|      0.0|     0.0|
|ENST00000492842.2...|   939|          812.0|      0.0|     0.0|
|ENST00000641515.2...|  2

# create master count spark data frame

In [11]:
sampleNamesLst = [None] * len(fileLst)
for i in range( len(fileLst) ):
    fileName = fileLst[i]
    sampleNamesLst[i] = fileName.replace(".quant.sf", "") 
    sampleNamesLst[i] = sampleNamesLst[i].replace(".", "_")
sampleNamesLst

['ctrl_1', 'ctrl_2', 'ctrl_3', 'kras_1', 'kras_2', 'kras_3']

In [12]:
# initialize master counts spark data fram
firstSampleName = sampleNamesLst[0]
masterCountSDF = quantSDFs[0].select( ["Name", "NumReads"] ).withColumnRenamed( "NumReads", firstSampleName )

# Register the DataFrame as a SQL temporary view
masterCountSDF.createOrReplaceTempView("masterCount")

In [13]:
for i in range( 1, len(sampleNamesLst)):
    sampleName = sampleNamesLst[i]
    #print(sampleName)
    
    # select the key and counts from the sample. 
    sampleSDF = quantSDFs[i].select( ["Name", "NumReads", ] ).withColumnRenamed( "NumReads", sampleName )
    sampleSDF.createOrReplaceTempView("sample")
    
    sqlStmt = ' select mc.*, {}  \n\
                      from \n\
                          masterCount as mc, \n\
                          sample  \n\
                      where \n\
                          mc.Name == sample.Name  \n'.format(sampleName)
    #print(sqlStmt)
    
    masterCountSDF = spark.sql( sqlStmt )
    masterCountSDF.createOrReplaceTempView("masterCount")
    #masterCountSDF.show()

# display the query plan. 
Spark does a lot of optimization under the cover

spark has several join strategies. I list 2 common ones bellow

I let it pick what ever it thought was best. I plan to check out query plan on
real data before causing it to execute

## Broadcast Hash Join
Also known as a map-side-only join, the broadcast hash join is employed when two
data sets, one small (fitting in the driver’s and executor’s memory) and another large
enough to ideally be spared from movement, need to be joined over certain conditions
or columns.

## Shuffle Sort Merge Join
The sort-merge algorithm is an efficient way to merge two large data sets over a common
key that is sortable, unique, and can be assigned to or stored in the same partition—
that is, two data sets with a common hashable key that end up being on the
same partition. From Spark’s perspective, this means that all rows within each data set
with the same key are hashed on the same partition on the same executor.

In [14]:
masterCountSDF.explain( mode='formatted' )

== Physical Plan ==
* Project (33)
+- * BroadcastHashJoin Inner BuildRight (32)
   :- * Project (27)
   :  +- * BroadcastHashJoin Inner BuildRight (26)
   :     :- * Project (21)
   :     :  +- * BroadcastHashJoin Inner BuildRight (20)
   :     :     :- * Project (15)
   :     :     :  +- * BroadcastHashJoin Inner BuildRight (14)
   :     :     :     :- * Project (9)
   :     :     :     :  +- * BroadcastHashJoin Inner BuildRight (8)
   :     :     :     :     :- * Project (3)
   :     :     :     :     :  +- * Filter (2)
   :     :     :     :     :     +- Scan csv  (1)
   :     :     :     :     +- BroadcastExchange (7)
   :     :     :     :        +- * Project (6)
   :     :     :     :           +- * Filter (5)
   :     :     :     :              +- Scan csv  (4)
   :     :     :     +- BroadcastExchange (13)
   :     :     :        +- * Project (12)
   :     :     :           +- * Filter (11)
   :     :     :              +- Scan csv  (10)
   :     :     +- BroadcastExchange (19)

# invoke an action
This will cause the optimized query plan to run

In [15]:
masterCountSDF.show()

+--------------------+------+------+------+------+------+------+
|                Name|ctrl_1|ctrl_2|ctrl_3|kras_1|kras_2|kras_3|
+--------------------+------+------+------+------+------+------+
|ENST00000456328.2...|   0.0| 2.006| 1.998|   0.0|   4.0|   0.0|
|ENST00000450305.2...|   0.0|   0.0|   0.0|   0.0|   0.0|   0.0|
|ENST00000488147.1...|   0.0|   0.0|   0.0|   0.0|   0.0|   0.0|
|ENST00000619216.1...|   0.0|   0.0|   0.0|   0.0|   0.0|   0.0|
|ENST00000473358.1...|   0.0|   0.0|   0.0|   0.0|   0.0|   0.0|
|ENST00000469289.1...|   0.0|   0.0|   0.0|   0.0|   0.0|   0.0|
|ENST00000607096.1...|   0.0|   0.0|   0.0|   0.0|   0.0|   0.0|
|ENST00000417324.1...|   0.0|   0.0|   0.0|   0.0|   0.0|   0.0|
|ENST00000461467.1...|  4.93|   0.0|   0.0|   0.0|   0.0|14.346|
|ENST00000606857.1...|   0.0|   0.0|   0.0|   0.0|   0.0|   0.0|
|ENST00000642116.1...|   0.0|   0.0|   0.0|   0.0|   0.0|   0.0|
|ENST00000492842.2...|   0.0|   0.0|   0.0|   0.0|   0.0|   0.0|
|ENST00000641515.2...|   

# Save master count

In [16]:
rootDir = Path.Path('./output')
rootDir.mkdir(exist_ok=True)

In [17]:
masterOutFile = rootDir.joinpath( "masterCount.tsv" )
# write to a single file
masterCountSDF.write.csv( masterOutFile.as_posix(), mode='overwrite', sep='\t', header=True)

In [18]:
# write to 3 files
masterPartsOutFile = rootDir.joinpath( "masterCountParts.tsv" )

numFiles = 3
masterCountSDF.repartition(numFiles) \
    .write \
    .csv( masterPartsOutFile.as_posix(), mode='overwrite', sep='\t', header=True)

masterCountSDF.show()

+--------------------+------+------+------+------+------+------+
|                Name|ctrl_1|ctrl_2|ctrl_3|kras_1|kras_2|kras_3|
+--------------------+------+------+------+------+------+------+
|ENST00000456328.2...|   0.0| 2.006| 1.998|   0.0|   4.0|   0.0|
|ENST00000450305.2...|   0.0|   0.0|   0.0|   0.0|   0.0|   0.0|
|ENST00000488147.1...|   0.0|   0.0|   0.0|   0.0|   0.0|   0.0|
|ENST00000619216.1...|   0.0|   0.0|   0.0|   0.0|   0.0|   0.0|
|ENST00000473358.1...|   0.0|   0.0|   0.0|   0.0|   0.0|   0.0|
|ENST00000469289.1...|   0.0|   0.0|   0.0|   0.0|   0.0|   0.0|
|ENST00000607096.1...|   0.0|   0.0|   0.0|   0.0|   0.0|   0.0|
|ENST00000417324.1...|   0.0|   0.0|   0.0|   0.0|   0.0|   0.0|
|ENST00000461467.1...|  4.93|   0.0|   0.0|   0.0|   0.0|14.346|
|ENST00000606857.1...|   0.0|   0.0|   0.0|   0.0|   0.0|   0.0|
|ENST00000642116.1...|   0.0|   0.0|   0.0|   0.0|   0.0|   0.0|
|ENST00000492842.2...|   0.0|   0.0|   0.0|   0.0|   0.0|   0.0|
|ENST00000641515.2...|   

In [19]:
masterCountSDF.rdd.getNumPartitions()

1